In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

from xxchallenge import *

from torchlensmaker.elements.optical_surfaces import CollisionSurface
from torchlensmaker.core.physics import reflection

# Parameters
A = tlm.parameter(-0.10)

# Primary mirror
primary = tlm.Parabola(1800, A=A, normalize=True)

cylinder2 = tlm.ImplicitCylinder(torch.tensor(0.), torch.tensor(22.5), torch.tensor(150.))

# Optical model
optics = tlm.Sequential(
    tlm.Gap(-1000),
    #XXLightSource.load(),
    tlm.PointSourceAtInfinity(1200),
    tlm.Gap(998),
    tlm.ReflectiveSurface(primary),
    tlm.Gap(-800),
    NonImagingRod(cylinder2),
)

xxrender(optics, sampling={"xx": 500, "letter": "both", "base": 500}, end=200)

In [ ]:
sampling = {"xx": 50, "disable_viewer": True, "letter": "both", "base": 50}

output = optics(tlm.default_input(sampling, dim=3, dtype=torch.float64))
print(output.loss)

In [ ]:
param_groups = [
    {'params': [A], 'lr': 1e-4},
]

record = tlm.optimize(
    optics,
    optimizer = optim.SGD(param_groups),
    sampling = {"xx": 50, "disable_viewer": True, "letter": "both", "base": 50},
    dim = 3,
    num_iter = 20
)


plot_record(record, param_groups, optics)
record.best()

settobest(optics, record, param_groups)

print("A", A)
F = 1./(primary._sag.unnorm(primary.diameter / 2) * 4)
print("F", F.item())

xxrender(optics, sampling={"xx": 50, "letter": "both", "base": 50})